# Data Processing of Test Data

In [3]:
# import
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import os
import json
import random

# importing in contracts from CRPWarner

path_groundTruth_excel = '../test-contracts/RugPull/dataset/groundTruth/groundTruth.xlsx'
df_groundTruth = pd.read_excel(path_groundTruth_excel, engine='openpyxl')

list_groundTruth_address = df_groundTruth['address'].tolist()
print('Number of groundTruth addresses: ', len(list_groundTruth_address))

# Specify folder paths
path_hex_folder = '../test-contracts/RugPull/dataset/groundTruth/hex'
path_sol_folder = '../test-contracts/RugPull/dataset/groundTruth/sol'

# Get all filenames in the folders without extensions
list_filenames_hex = [os.path.splitext(f)[0] for f in os.listdir(path_hex_folder) if os.path.isfile(os.path.join(path_hex_folder, f))]
list_filenames_sol = [os.path.splitext(f)[0] for f in os.listdir(path_sol_folder) if os.path.isfile(os.path.join(path_sol_folder, f))]


# Find missing hex and sol files
def get_missing_addresses(ground_truth_addresses, existing_hex_files, existing_sol_files):
    missing_hex_files = []
    missing_sol_files = []
    for address in ground_truth_addresses:
        if address not in existing_hex_files:
            missing_hex_files.append(address)
        if address not in existing_sol_files:
            missing_sol_files.append(address)
    return missing_hex_files, missing_sol_files


list_hex_lack, list_sol_lack = get_missing_addresses(
    list_groundTruth_address, list_filenames_hex, list_filenames_sol)
print(
    f"Missing Addresses are: \n hex:{list_hex_lack}, \n sol: {list_sol_lack}")
print('The number of missing addresses: hex: ', len(
    list_hex_lack), '; sol: ', len(list_sol_lack))

# Setup of API keys

# Load environment variables
load_dotenv()

# Initialize API keys
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

if not OPENAI_API_KEY or not ANTHROPIC_API_KEY:
    raise ValueError("API keys for OpenAI and Anthropic must be set in the .env file.")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

# Sample Contract for testing

In [7]:
# Define the smart contract code to be audited (example contract code snippet)
smart_contract_code = """
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract ExampleToken {
    string public name = "ExampleToken";
    string public symbol = "EXT";
    uint8 public decimals = 18;
    uint256 public totalSupply;
    address public owner;

    mapping(address => uint256) public balanceOf;
    mapping(address => mapping(address => uint256)) public allowance;

    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);

    constructor(uint256 _initialSupply) {
        owner = msg.sender;
        totalSupply = _initialSupply * (10 ** uint256(decimals));
        balanceOf[owner] = totalSupply;
        emit Transfer(address(0), owner, totalSupply);
    }

    function transfer(address _to, uint256 _value) public returns (bool success) {
        require(balanceOf[msg.sender] >= _value, "Insufficient balance");
        balanceOf[msg.sender] -= _value;
        balanceOf[_to] += _value;
        emit Transfer(msg.sender, _to, _value);
        return true;
    }

    function approve(address _spender, uint256 _value) public returns (bool success) {
        allowance[msg.sender][_spender] = _value;
        emit Approval(msg.sender, _spender, _value);
        return true;
    }

    function transferFrom(address _from, address _to, uint256 _value) public returns (bool success) {
        require(_value <= balanceOf[_from], "Insufficient balance");
        require(_value <= allowance[_from][msg.sender], "Allowance exceeded");
        balanceOf[_from] -= _value;
        balanceOf[_to] += _value;
        allowance[_from][msg.sender] -= _value;
        emit Transfer(_from, _to, _value);
        return true;
    }
}
"""

# GPT 4-O

## Running a simple 3 experiment using OPENAI

In [5]:
# Import necessary libraries
from openai import ChatCompletion, OpenAI
import json
import openai
import os
from dotenv import load_dotenv


def open_ai_audit_smart_contract(smart_contract_code):
    # Load environment variables
    load_dotenv()
    api_key = os.environ.get('OPENAI_API_KEY')

    if not api_key:
        raise ValueError("OPENAI_API_KEY not found in environment variables")

    # Initialize the OpenAI client
    client = OpenAI(api_key=api_key)

    # Define the prompt
    prompt = f"""
    You are an AI model specialized in auditing smart contracts for potential vulnerabilities, particularly rug pull scenarios. Your task is to analyze the provided smart contract and identify any code patterns or logic flaws that could allow the contract creators to execute a rug pull.

    A rug pull is a type of exit scam where the developers of a cryptocurrency project suddenly withdraw all funds from the liquidity pool, leaving investors with worthless tokens. Common indicators of rug pull vulnerabilities include, but are not limited to:

    - Centralized control over contract functions.
    - The ability to mint or burn tokens without restrictions.
    - Unverified external contracts or dependencies.
    - Lack of time locks on critical functions.
    - Absence of multi-signature requirements for sensitive operations.

    Given the following smart contract's code, highlight the specific lines or sections that might be exploited for a rug pull. Provide detailed explanations for each identified vulnerability and suggest possible mitigations or improvements to enhance the contract's security.

    Smart Contract Code:
    {smart_contract_code}
    """

    try:
        # Make the API call
        response = client.chat.completions.create(
            model="gpt-4",  # Note: Changed from "gpt-4o" to "gpt-4"
            messages=[
                {"role": "system", "content": "You are a smart contract auditor, skilled in identifying vulnerabilities in smart contract code."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=20000
        )

        # Extract and return the response text
        return response.choices[0].message.content

    except Exception as e:
        return f"An error occurred during the audit: {str(e)}"

In [6]:
output_file = "audit_reports/audit_result.txt"

# Call the function with the example contract
audit_result = open_ai_audit_smart_contract(example_contract)
# Optionally, save the response to a file
with open(output_file, "w") as file:
    file.write(response_text)

NameError: name 'audit_smart_contract' is not defined

# Claude 3.5 Sonnet

In [11]:
# Import necessary libraries
import os
import anthropic

output_file = "audit_reports/claude_audit_result.txt"
def claude_audit_smart_contract(smart_contract_code):
    load_dotenv()

    # Set the API key using an environment variable
    api_key = os.environ.get('ANTHROPIC_API_KEY')
    if not api_key:
        raise ValueError("ANTHROPIC_API_KEY environment variable is not set")

    # Initialize the Anthropic client
    client = anthropic.Anthropic(api_key=api_key)

    # Define the prompt
    prompt = f"""
    You are an AI model specialized in auditing smart contracts for potential vulnerabilities, particularly rug pull scenarios. Your task is to analyze the provided smart contract and identify any code patterns or logic flaws that could allow the contract creators to execute a rug pull.

    A rug pull is a type of exit scam where the developers of a cryptocurrency project suddenly withdraw all funds from the liquidity pool, leaving investors with worthless tokens. Common indicators of rug pull vulnerabilities include, but are not limited to:

    - Centralized control over contract functions.
    - The ability to mint or burn tokens without restrictions.
    - Unverified external contracts or dependencies.
    - Lack of time locks on critical functions.
    - Absence of multi-signature requirements for sensitive operations.

    Given the following smart contract's code, highlight the specific lines or sections that might be exploited for a rug pull. Provide detailed explanations for each identified vulnerability and suggest possible mitigations or improvements to enhance the contract's security.

    Smart Contract Code:
    {smart_contract_code}
    """

    # Make the API call
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=4096,
        temperature=0.7,
        system="You are a smart contract auditor, skilled in identifying vulnerabilities in smart contract code.",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    # Get the response text
    audit_result = response.content[0].text

    # Write the response to the output file
    with open(output_file, 'w') as f:
        f.write(audit_result)

    print(f"Audit result has been written to {output_file}")

In [12]:
contract = claude_audit_smart_contract(smart_contract_code)

Audit result has been written to audit_reports/audit_result.txt


# Experiment

## Setup

In [ ]:
# Functions for Setup

def load_prompts():
    """Load prompts from the prompts directory."""
    prompts = {}
    prompt_dir = Path('notebooks/prompts/')
    basic_prompt_path = prompt_dir / 'basic_ai_prompt.txt'
    developer_meta_prompt_path = prompt_dir / 'developer_meta_prompt.txt'

    with open(basic_prompt_path, 'r') as file:
        prompts['basic'] = file.read()

    with open(developer_meta_prompt_path, 'r') as file:
        prompts['developer_meta'] = file.read()

    return prompts


def load_contracts(contract_dir='../test-contracts/RugPull/dataset/groundTruth/'):
    """Load smart contracts from the specified directory."""
    contracts = []
    for file in os.listdir(contract_dir):
        if file.endswith('.sol'):
            with open(os.path.join(contract_dir, file), 'r') as f:
                contracts.append({'filename': file, 'code': f.read()})
    return contracts


def initialize_openai_client():
    """Initialize the OpenAI client."""
    return openai.ChatCompletion()


def initialize_anthropic_client():
    """Initialize the Anthropic client."""
    return anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)


def audit_contract_openai(model, prompt, contract_code):
    """Audit a single contract using OpenAI's GPT-4-O model."""
    client = initialize_openai_client()
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a smart contract auditor."},
                {"role": "user", "content": prompt.replace(
                    "[Insert smart contract code here]", contract_code)}
            ],
            temperature=0.7,
            max_tokens=2000
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"


def audit_contract_anthropic(model, prompt, contract_code):
    """Audit a single contract using Anthropic's Claude 3.5 Sonnet model."""
    client = initialize_anthropic_client()
    try:
        response = client.completions.create(
            model=model,
            prompt=prompt.replace(
                "[Insert smart contract code here]", contract_code),
            max_tokens=3000,
            temperature=0.7
        )
        return response.completion
    except Exception as e:
        return f"Error: {str(e)}"


def audit_contract_openai(model, prompt, contract_code):
    """Audit a single contract using OpenAI's GPT-4-O model."""
    client = initialize_openai_client()
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a smart contract auditor."},
                {"role": "user", "content": prompt.replace(
                    "[Insert smart contract code here]", contract_code)}
            ],
            temperature=0.7,
            max_tokens=2000
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"


def audit_contract_anthropic(model, prompt, contract_code):
    """Audit a single contract using Anthropic's Claude 3.5 Sonnet model."""
    client = initialize_anthropic_client()
    try:
        response = client.completions.create(
            model=model,
            prompt=prompt.replace(
                "[Insert smart contract code here]", contract_code),
            max_tokens=3000,
            temperature=0.7
        )
        return response.completion
    except Exception as e:
        return f"Error: {str(e)}"
    

def evaluate_audit(audit_result, known_errors):
    """
    Evaluate whether the audit_result contains the known_errors.
    
    Parameters:
    - audit_result (str): The text output from the audit model.
    - known_errors (list): A list of known error descriptions.
    
    Returns:
    - bool: True if all known errors are found, False otherwise.
    """
    audit_result_lower = audit_result.lower()
    return all(error.lower() in audit_result_lower for error in known_errors)


def analyze_results(results, total_contracts=20):
    """Analyze the audit results."""
    analysis = {}
    for model_name in results:
        analysis[model_name] = {}
        for prompt_type in results[model_name]:
            successes = sum(
                1 for result in results[model_name][prompt_type] if result['found_errors'])
            failure = total_contracts - successes
            analysis[model_name][prompt_type] = {
                'Success': successes,
                'Failure': failure,
                'Success Rate (%)': round((successes / total_contracts) * 100, 2)
            }
    return analysis

## Performing the audit

In [ ]:
# Load prompts and contracts
prompts = load_prompts()
contracts = load_contracts()

# Define models
models = {
    'GPT4O': 'gpt-4',
    'Claude3.5_Sonnet': 'claude-3-5-sonnet-20240620'
}

# Define prompt types
prompt_types = ['basic', 'developer_meta']

# Initialize results storage
results = {
    'GPT4O': {'basic': [], 'developer_meta': []},
    'Claude3.5_Sonnet': {'basic': [], 'developer_meta': []}
}

# Iterate over each contract
for contract in contracts[:20]:  # Ensure only 20 contracts are processed
    filename = contract['filename']
    code = contract['code']

    # Retrieve known errors for this contract
    # Assuming df_groundTruth has 'address' and 'errors' columns
    # Adjust the column names based on your actual DataFrame
    known_errors = df_groundTruth[df_groundTruth['address']
                                  == filename]['errors'].tolist()

    for model_name, model_identifier in models.items():
        for prompt_type in prompt_types:
            if model_name == 'GPT4O':
                audit_result = audit_contract_openai(
                    model_identifier, prompts[prompt_type], code)
            elif model_name == 'Claude3.5_Sonnet':
                audit_result = audit_contract_anthropic(
                    model_identifier, prompts[prompt_type], code)
            else:
                audit_result = "Unsupported model."

            # Evaluate the audit result
            found_errors = evaluate_audit(audit_result, known_errors)

            # Store the result
            results[model_name][prompt_type].append({
                'contract': filename,
                'found_errors': found_errors
            })

            print(
                f"Completed {model_name} with {prompt_type} on {filename}: {'Success' if found_errors else 'Failure'}")

### Saving the results to CSV

In [ ]:
# Create audit_reports directory if it doesn't exist
os.makedirs('audit_reports', exist_ok=True)

# Save the results to CSV files
for model_name in results:
    for prompt_type in results[model_name]:
        df_results = pd.DataFrame(results[model_name][prompt_type])
        csv_path = f'audit_reports/{model_name.lower()}_{prompt_type}_results.csv'
        df_results.to_csv(csv_path, index=False)
        print(f"Results saved to {csv_path}")

### Analyzing the results

In [ ]:
def analyze_results(results, total_contracts=20):
    """Analyze the audit results."""
    analysis = {}
    for model_name in results:
        analysis[model_name] = {}
        for prompt_type in results[model_name]:
            successes = sum(
                1 for result in results[model_name][prompt_type] if result['found_errors'])
            failure = total_contracts - successes
            analysis[model_name][prompt_type] = {
                'Success': successes,
                'Failure': failure,
                'Success Rate (%)': round((successes / total_contracts) * 100, 2)
            }
    return analysis


# Perform analysis
analysis = analyze_results(results)

# Display the analysis
for model, prompt_data in analysis.items():
    print(f"\nModel: {model}")
    for prompt, data in prompt_data.items():
        print(f"  Prompt: {prompt}")
        for key, value in data.items():
            print(f"    {key}: {value}")